In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/인천e음_통합_일별.csv',encoding= 'utf-8')

In [ ]:
import pandas as pd
import numpy as np

# 정책조합 함수 (연매출 3억 이하, 3억~30억 대상 비율 튜플로 반환)
def get_policy_status(row):
    return (row['연매출 3억원 이하'], row['연매출 3억~30억원'])

df['정책조합'] = df.apply(get_policy_status, axis=1)
df['연월'] = df['일자'].dt.to_period('M')
df['연도'] = df['일자'].dt.year

# 연도별 실제 집행액 (예산)
budget_dict = {
    2020: 214037000000,
    2021: 344411000000,
    2022: 304400000000,
    2023: 180991000000,
    2024: 129410000000,
}

budget_df = pd.DataFrame(list(budget_dict.items()), columns=['연도', '실제집행액'])
budget_df['월별투자비용'] = budget_df['실제집행액'] / 12

# 캐시백 비율 선택 함수 (target='low' or 'mid')
def get_cashback_ratio(row, target='low'):
    ratio_str = row['정책조합'][0] if target == 'low' else row['정책조합'][1]
    return float(ratio_str.replace('%', '')) / 100

# 결과 저장용 리스트
results = []

# 분석 대상: low = 3억 이하, mid = 3억~30억
target_group = 'mid'  # 'low'로 바꾸면 3억 이하 대상

for region, group in df.groupby('지역'):
    monthly_sum = group.groupby(['연월', '정책조합']).agg({'전체금액': 'sum'}).reset_index()
    monthly_sum['지역'] = region
    monthly_sum['연도'] = monthly_sum['연월'].dt.year

    # 주요 정책조합(비중 > 90%) 필터링
    total_by_month = group.groupby(['연월', '정책조합'])['전체금액'].sum().reset_index()
    total_by_month['월_총합'] = total_by_month.groupby('연월')['전체금액'].transform('sum')
    total_by_month['비중'] = total_by_month['전체금액'] / total_by_month['월_총합']
    dominant_df = total_by_month[total_by_month['비중'] > 0.9][['연월', '정책조합']]

    monthly_df = pd.merge(monthly_sum, dominant_df, on=['연월', '정책조합'])
    monthly_df.sort_values('연월', inplace=True)
    monthly_df['연월'] = monthly_df['연월'].astype(str)
    monthly_df['전월금액'] = monthly_df['전체금액'].shift(1)
    monthly_df['전월_정책조합'] = monthly_df['정책조합'].shift(1)

    monthly_df['정책_개입'] = (
        (monthly_df['정책조합'] != ('0%', '0%')) &
        ((monthly_df['전월_정책조합'] == ('0%', '0%')) |
         (monthly_df['정책조합'] != monthly_df['전월_정책조합']))
    )

    monthly_df = monthly_df.merge(budget_df[['연도', '월별투자비용']], on='연도', how='left')

    # 캐시백 비율 적용
    monthly_df['캐시백비율'] = monthly_df.apply(lambda row: get_cashback_ratio(row, target=target_group), axis=1)
    monthly_df['캐시백소진액'] = monthly_df['전체금액'] * monthly_df['캐시백비율']

    # ROI 계산
    monthly_df['매출증가액'] = monthly_df['전체금액'] - monthly_df['전월금액']
    monthly_df['ROI_전체'] = monthly_df['매출증가액'] / monthly_df['캐시백소진액']
    monthly_df['ROI_정책효과'] = monthly_df['ROI_전체'].where(monthly_df['정책_개입'])

    results.append(monthly_df)

final_df = pd.concat(results, ignore_index=True)

def show_region_data(region_name):
    #cols = ['지역', '연월', '정책조합', '전체금액', '전월금액', '캐시백비율', '캐시백소진액', '정책_개입', 'ROI_정책효과']
    cols = ['지역', '연월', '정책조합','ROI_전체', '정책_개입', 'ROI_정책효과']
    region_data = final_df[final_df['지역'] == region_name]
    return region_data[cols].reset_index(drop=True)

# 사용 예시: 인천광역시 중구 데이터 보기
result = show_region_data("인천광역시 옹진군")
print(result)


In [ ]:
# 지역별 ROI_정책효과 통계 계산
region_sensitivity = final_df[final_df['정책_개입']].groupby('지역').agg(
    정책개입횟수=('정책_개입', 'sum'),
    정책효과평균=('ROI_정책효과', 'mean'),
    정책효과절대값평균=('ROI_정책효과', lambda x: x.abs().mean())
).reset_index()

# 민감도 평가 기준 추가 (절대값 기준)
def classify_sensitivity(value):
    if value >= 3:
        return '🔥 매우 민감'
    elif value >= 2:
        return '⚠️ 중간~높음'
    elif value >= 1:
        return '✅ 보통 수준'
    else:
        return '✅ 매우 둔감'

region_sensitivity['민감도'] = region_sensitivity['정책효과절대값평균'].apply(classify_sensitivity)

# 보기 좋게 정렬
region_sensitivity = region_sensitivity.sort_values(by='정책효과절대값평균', ascending=False)

# 결과 확인
print(region_sensitivity)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (윈도우 사용자용 예시: 'Malgun Gothic')
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 그래프 크기 및 스타일
plt.figure(figsize=(12, 6))
sns.barplot(
    data=region_sensitivity,
    x='정책효과절대값평균',
    y='지역',
    hue='민감도',
    dodge=False,
    palette='coolwarm'
)
plt.title('지역별 캐시백 정책 민감도 (절대 ROI 기준)')
plt.xlabel('정책효과 절대값 평균 (|ROI|)')
plt.ylabel('지역')
plt.legend(title='민감도', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
